In [4]:

import pandas as pd
import requests
from io import StringIO
from datetime import datetime
import argparse
import os
from site_identifier import identify_sites

In [5]:
# Configuration
GOOGLE_SHEET_URL = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRpva-TXUaQR_6tJoXX2vnSN2ertC5GNxAgssqmXvIhqHBNrscDxSxtiSWbCiiHqAoSHb3SzXDQw_VX/pub?gid=1048590026&single=true&output=csv"

In [6]:
"""Fetch Google Sheets data"""
print("Fetching data from Google Sheets...")
response = requests.get(GOOGLE_SHEET_URL, timeout=30)
df = pd.read_csv(StringIO(response.text), header=None, low_memory=False)

Fetching data from Google Sheets...


In [7]:
sites = identify_sites(df)

Identifying sites dynamically...
Strategy: Finding 'INV. SETTING' labels and extracting site names

  ✓ Row 4: OLD Morongo | Cabazon
  ✓ Row 43: NEW Morongo Site #2
  ✓ Row 93: Fort Independence
  ✓ Row 127: Campo | Golden Acorn
  ✓ Row 168: Bishop | Pauite Palace
  ✓ Row 209: Bishop Pauite 2 Nobi
  ✓ Row 248: Pechanga | Temecula
  ✓ Row 283: Salton Sea | Red Earth
  ✓ Row 350: Cahuilla | Anza
  ✓ Row 385: La Jolla Trading Post
  ✓ Row 420: Eagle Feather | Tule River
  ✓ Row 455: Sycuan | El Cajon
  ✓ Row 490: Rincon | Valley Center
  ✓ Row 525: Santa Rosa Pit Stop
  ✓ Row 560: Fort Mojave Smoke Shop
  ✓ Row 599: Thalypo | Fort Mojave
  ✓ Row 666: Pala
  ✓ Row 701: Shivwits | Utah
  ✓ Row 762: San Pasqual | Valley View
  ✓ Row 905: Barona
  ✓ Row 944: Kanosh | Pahvant Travel
  ✓ Row 995: Santa Ysabel
  ✓ Row 1029: Palms | Coachella
  ✓ Row 1112: Kaibab | Red Cliffs
  ✓ Row 1147: Viejas | Alpine
  ✓ Row 1197: Chumash

✓ Total sites identified: 26


In [8]:
def get_all_dates(df, start_col=6):
    """Extract all date columns from the sheet (up to today only)"""
    print("\nExtracting all dates...")
    dates_row = df.iloc[0, start_col:]
    
    today = pd.Timestamp.now().normalize()  # Get today's date at midnight
    
    date_data = []
    for col_idx, date_val in enumerate(dates_row, start=start_col):
        if pd.notna(date_val):
            try:
                parsed = pd.to_datetime(str(date_val), format='%b-%d-%y', errors='coerce')
                if parsed and parsed <= today:  # Only include dates up to today
                    date_data.append((col_idx, parsed))
            except:
                pass
    
    print(f"✓ Found {len(date_data)} dates (up to today)")
    if date_data:
        print(f"  Date range: {date_data[0][1].date()} to {date_data[-1][1].date()}")
    return date_data

In [9]:
all_dates = get_all_dates(df)


Extracting all dates...
✓ Found 610 dates (up to today)
  Date range: 2024-03-01 to 2025-10-31


In [10]:
def extract_site_readings(df, site_row, site_name, date_columns):
    """Extract readings for a single site"""
    print(f"  Extracting READINGS for {site_name}...")
    
    # Find READINGS section
    reading_start_row = None
    reading_end_row = None
    
    for offset in range(20):
        row_idx = site_row + offset
        if row_idx >= len(df):
            break
        
        section_label = str(df.iloc[row_idx, 3]).strip() if pd.notna(df.iloc[row_idx, 3]) else ""
        
        if "READINGS" in section_label.upper():
            reading_start_row = row_idx + 1
            break
    
    if reading_start_row is None:
        return []
    
    # Find end of READINGS section
    for offset in range(15):
        row_idx = reading_start_row + offset
        if row_idx >= len(df):
            break
        
        section_label = str(df.iloc[row_idx, 3]).strip() if pd.notna(df.iloc[row_idx, 3]) else ""
        
        if any(keyword in section_label.upper() for keyword in ['ULLAGE', 'LOADS', 'CARRIER', 'NOTES']):
            reading_end_row = row_idx
            break
    
    if reading_end_row is None:
        reading_end_row = reading_start_row + 10
    
    # Scan READINGS section
    records = []
    products_found = {}
    
    for row_idx in range(reading_start_row, reading_end_row):
        if row_idx >= len(df):
            break
        
        product_cell = df.iloc[row_idx, 4]
        
        if pd.notna(product_cell):
            product = str(product_cell).strip()
            
            for key in ['87', '88', '91', 'dsl', 'racing', 'red', 'total']:
                if product.lower().startswith(key):
                    if product not in products_found:
                        products_found[product] = []
                    products_found[product].append(row_idx)
    
    # Extract readings for each date
    for col_idx, date in date_columns:
        for product, row_indices in products_found.items():
            record = {
                'Date': date.strftime('%Y-%m-%d'),
                'Site': site_name,
                'Product': product
            }
            
            for tank_num, row_idx in enumerate(row_indices, start=1):
                value = df.iloc[row_idx, col_idx]
                
                if pd.notna(value):
                    try:
                        clean_val = str(value).replace(',', '').strip()
                        numeric_val = float(clean_val) if clean_val else None
                        record[f'Tank_{tank_num}_Reading'] = numeric_val
                    except:
                        record[f'Tank_{tank_num}_Reading'] = None
                else:
                    record[f'Tank_{tank_num}_Reading'] = None
            
            records.append(record)
    
    return records


In [11]:

def extract_site_sales_actual(df, site_row, site_name, date_columns):
    """Extract actual sales for a single site"""
    print(f"  Extracting SALES (actual) for {site_name}...")
    
    # Find SALES (actual) section
    sales_start_row = None
    
    for offset in range(200):
        row_idx = site_row + offset
        if row_idx >= len(df):
            break
        
        col1_label = str(df.iloc[row_idx, 1]).strip() if pd.notna(df.iloc[row_idx, 1]) else ""
        col3_label = str(df.iloc[row_idx, 3]).strip() if pd.notna(df.iloc[row_idx, 3]) else ""
        
        # Look for SALES (actual) specifically, not SALES (projected)
        if "SALES" in col1_label.upper() and "ACTUAL" in col1_label.upper():
            sales_start_row = row_idx
            break
        elif "ACTUAL" in col3_label.upper() and "SALES" in col1_label.upper():
            sales_start_row = row_idx
            break
    
    if sales_start_row is None:
        return []
    
    # Extract products in SALES (actual) section
    records = []
    products_found = {}
    
    for row_idx in range(sales_start_row, sales_start_row + 10):
        if row_idx >= len(df):
            break
        
        product_cell = df.iloc[row_idx, 4]
        
        if pd.notna(product_cell):
            product = str(product_cell).strip()
            if "READING" in product.upper():
                # print(f"    Reached end of products at row {row_idx}.")
                break
            # Get base product (87, 88, racing, red, 91, dsl) - include totals
            base_product = None
            is_total = False
            
            if "87" in product:
                base_product = product
                is_total = "total" in product.lower()
            elif "88" in product:
                base_product = product
                is_total = "total" in product.lower()
            elif "91" in product:
                base_product = product
                is_total = "total" in product.lower()
            elif "dsl" in product.lower():
                base_product = product
                is_total = "total" in product.lower()
            elif "racing" in product.lower():
                base_product = product
                is_total = "total" in product.lower()
            elif "red" in product.lower():
                base_product = product
                is_total = "total" in product.lower()


            if base_product:
                products_found[product] = {
                    'row_idx': row_idx,
                    'base_product': base_product,
                    'is_total': is_total
                }
    
    # Extract sales for each date
    for col_idx, date in date_columns:
        for product_key, product_info in products_found.items():
            row_idx = product_info['row_idx']
            value = df.iloc[row_idx, col_idx]
            
            if pd.notna(value):
                try:
                    clean_val = str(value).replace(',', '').strip()
                    sales_val = float(clean_val) if clean_val else None
                    
                    if sales_val is not None:
                        records.append({
                            'Date': date.strftime('%Y-%m-%d'),
                            'Site': site_name,
                            'Product': product_info['base_product'],
                            'Sales_Actual': sales_val,
                            'Is_Total': product_info['is_total']
                        })
                except:
                    pass
    
    return records


In [28]:
def get_three_week_avg(df, site_row, site_name, all_dates):    
    """Get 3-week average sales for a site"""
    print(f" Getting 3-week average sales for {site_name}...")
    avg_start_row = None
    product_name = None
    avg_end_row = None
    for offset in range(200):
        row_idx = site_row + offset
        if row_idx >= len(df):
            break
        
        section_label = str(df.iloc[row_idx-1, 3]).strip() if pd.notna(df.iloc[row_idx-1, 3]) else ""
        avg_col = str(df.iloc[row_idx, 4]).strip() if pd.notna(df.iloc[row_idx, 4]) else ""


        if "3 WK AVG" in avg_col.upper():
            avg_start_row = row_idx + 1
            break
    
    if avg_start_row is None:
        return []

    # Find end of Avg Section
    for offset in range(200):
        row_idx = avg_start_row + offset
        if row_idx >= len(df):
            break
        
        section_label = str(df.iloc[row_idx, 3]).strip() if pd.notna(df.iloc[row_idx, 3]) else ""
        
        if any(keyword in section_label.upper() for keyword in ['ACTUAL']):
            avg_end_row = row_idx
            break
    
    if avg_end_row is None:
        avg_end_row = avg_start_row + 100

    records = []
    products_found = {}

    for row_idx in range(avg_start_row, avg_end_row):
        if row_idx >= len(df):
            break
        
        product_cell = df.iloc[row_idx-2, 4]
        
        if pd.notna(product_cell):
            product = str(product_cell).strip()
            
            if any(key in product for key in ['87', '88', '91', 'dsl', 'racing', 'red']):
                if product not in products_found:
                    products_found[product] = []
                
                products_found[product].append(row_idx)

    for col_idx, date in all_dates:
        for product, row_indices in products_found.items():
            record = {
                'Date': date.strftime('%Y-%m-%d'),
                'Site': site_name,
                'Product': product
            }
            for tank_num, row_idx in enumerate(row_indices, start=1):
                value = df.iloc[row_idx-1, col_idx]
                if pd.notna(value):
                    try:
                        clean_val = str(value).replace(',', '').strip()
                        avg_val = float(clean_val) if clean_val else None
                        record[f'Tank_{tank_num}_3_Week_Avg'] = avg_val
                    except:
                        record[f'Tank_{tank_num}_3_Week_Avg'] = None
                else:
                    record[f'Tank_{tank_num}_3_Week_Avg'] = None
            
            records.append(record)

    return records

In [ ]:
def get_2_month_avg(df, site_row, site_name, all_dates):    
    """Get 2-month average sales for a site"""
    print(f" Getting 2-month average sales for {site_name}...")
    avg_start_row = None
    product_name = None
    avg_end_row = None
    for offset in range(200):
        row_idx = site_row + offset
        if row_idx >= len(df):
            break
        
        section_label = str(df.iloc[row_idx-1, 3]).strip() if pd.notna(df.iloc[row_idx-1, 3]) else ""
        avg_col = str(df.iloc[row_idx, 4]).strip() if pd.notna(df.iloc[row_idx, 4]) else ""


        if "2 MO AVG" in avg_col.upper():
            avg_start_row = row_idx + 1
            break
    
    if avg_start_row is None:
        return []

    # Find end of Avg Section
    for offset in range(200):
        row_idx = avg_start_row + offset
        if row_idx >= len(df):
            break
        
        section_label = str(df.iloc[row_idx, 3]).strip() if pd.notna(df.iloc[row_idx, 3]) else ""
        
        if any(keyword in section_label.upper() for keyword in ['ACTUAL']):
            avg_end_row = row_idx
            break
    
    if avg_end_row is None:
        avg_end_row = avg_start_row + 100

    records = []
    products_found = {}

    for row_idx in range(avg_start_row, avg_end_row):
        if row_idx >= len(df):
            break
        
        product_cell = df.iloc[row_idx-3, 4]
        
        if pd.notna(product_cell):
            product = str(product_cell).strip()
            
            if any(key in product for key in ['87', '88', '91', 'dsl', 'racing', 'red']):
                if product not in products_found:
                    products_found[product] = []
                
                products_found[product].append(row_idx)

    for col_idx, date in all_dates:
        for product, row_indices in products_found.items():
            record = {
                'Date': date.strftime('%Y-%m-%d'),
                'Site': site_name,
                'Product': product
            }
            for tank_num, row_idx in enumerate(row_indices, start=1):
                value = df.iloc[row_idx-1, col_idx]
                if pd.notna(value):
                    try:
                        clean_val = str(value).replace(',', '').strip()
                        avg_val = float(clean_val) if clean_val else None
                        record[f'Tank_{tank_num}_2_Month_Avg'] = avg_val
                    except:
                        record[f'Tank_{tank_num}_2_Month_Avg'] = None
                else:
                    record[f'Tank_{tank_num}_2_Month_Avg'] = None

            records.append(record)

    return records

In [ ]:
for site_row , site_name in sites:
    print(f"\n{site_name}:")

    print(f"  Extracting SALES (actual) for {site_name}...")
    sales_start_row = None

    for offset in range(40):
        row_idx = site_row + offset
        if row_idx >= len(df):
            break
        
        col1_label = str(df.iloc[row_idx, 1]).strip() if pd.notna(df.iloc[row_idx, 1]) else ""
        col3_label = str(df.iloc[row_idx, 3]).strip() if pd.notna(df.iloc[row_idx, 3]) else ""

        if "SALES" in col1_label.upper() and "ACTUAL" in col3_label.upper():
            sales_start_row = row_idx
            break
        elif "ACTUAL" in col3_label.upper() and 'SALES' in col1_label.upper():
            sales_start_row = row_idx
            break
    print(f"    Found SALES (actual) row at index {sales_start_row}")

    if sales_start_row is None:
        print(f"    ⚠️ SALES (actual) row not found for {site_name}, skipping...")
        break
    records = []
    products_found = {}

    for row_idx in range(sales_start_row, sales_start_row+10):
        if row_idx >= len(df):
            break

        product_cell = df.iloc[row_idx, 4]
        # print(f"    Checking row {row_idx}, product cell: {product_cell}")

        if pd.notna(product_cell):
            product = str(product_cell).strip()
            # print(f"    Found product: {product}")
            if "READING" in product.upper():
                # print(f"    Reached end of products at row {row_idx}.")
                break
            base_product = None
            is_total = False

            if "87" in product:
                base_product = product
                is_total = "total" in product.upper()
            elif "91" in product:
                base_product = product
                is_total = "total" in product.upper()
            elif "dsl" in product.lower():
                base_product = product
                is_total = "total" in product.lower()
            
            if base_product:
                products_found[product] = {
                    'row_idx': row_idx,
                    'base_product': base_product,
                    'is_total': is_total
                }

    print(f"    Products found so far: {products_found}")

    for col_idx, date in all_dates:
        for _, product_info in products_found.items():
            row_idx = product_info['row_idx']
            value = df.iloc[row_idx, col_idx]

            print(f"      Date: {date.date()}, Product: {product_info['base_product']}, Value: {value}, Col: {col_idx}, Row: {row_idx}")
    break


In [27]:
for site_row, site_name in sites:
    if site_name != "Salton Sea | Red Earth":
        continue
        """Get 3-week average sales for a site"""
    print(f" Getting 3-week average sales for {site_name}...")
    avg_start_row = None
    product_name = None
    avg_end_row = None
    for offset in range(200):
        row_idx = site_row + offset
        if row_idx >= len(df):
            break
        
        section_label = str(df.iloc[row_idx-1, 3]).strip() if pd.notna(df.iloc[row_idx-1, 3]) else ""
        avg_col = str(df.iloc[row_idx, 4]).strip() if pd.notna(df.iloc[row_idx, 4]) else ""


        if "3 WK AVG" in avg_col.upper():
            avg_start_row = row_idx + 1
            print(f"   Found 3 WK AVG at row {avg_start_row}")
            break
    
    if avg_start_row is None:
        print("   3 WK AVG section not found, skipping...")
        continue

 Getting 3-week average sales for Salton Sea | Red Earth...
   Found 3 WK AVG at row 313


In [29]:
all_readings = []
all_loads = []
all_tank_sizes = []
all_inv_settings = []
all_sales_actual = []
all_three_week_avg = []
all_2_month_avg = []
for site_row, site_name in sites:
    # readings = extract_site_readings(df, site_row, site_name, all_dates)
    # all_readings.extend(readings)
    # print(f"    ✓ {len(readings)} reading records")
    # # Extract sales actual
    # sales_actual = extract_site_sales_actual(df, site_row, site_name, all_dates)
    # all_sales_actual.extend(sales_actual)
    # print(f"    ✓ {len(sales_actual)} sales actual records")

    three_week_avg = get_three_week_avg(df, site_row, site_name, all_dates)
    all_three_week_avg.extend(three_week_avg)
    print(f"    ✓ {len(three_week_avg)} 3-week average records")

    two_month_avg = get_2_month_avg(df, site_row, site_name, all_dates)
    all_2_month_avg.extend(two_month_avg)

# df_readings = pd.DataFrame(all_readings)
# df_sales_actual = pd.DataFrame(all_sales_actual)
df_three_week_avg = pd.DataFrame(all_three_week_avg)
df_2_month_avg = pd.DataFrame(all_2_month_avg)

 Getting 3-week average sales for OLD Morongo | Cabazon...
    ✓ 1830 3-week average records
 Getting 2-month average sales for OLD Morongo | Cabazon...
 Getting 3-week average sales for NEW Morongo Site #2...
    ✓ 1830 3-week average records
 Getting 2-month average sales for NEW Morongo Site #2...
 Getting 3-week average sales for Fort Independence...
    ✓ 1830 3-week average records
 Getting 2-month average sales for Fort Independence...
 Getting 3-week average sales for Campo | Golden Acorn...
    ✓ 1830 3-week average records
 Getting 2-month average sales for Campo | Golden Acorn...
 Getting 3-week average sales for Bishop | Pauite Palace...
    ✓ 1830 3-week average records
 Getting 2-month average sales for Bishop | Pauite Palace...
 Getting 3-week average sales for Bishop Pauite 2 Nobi...
    ✓ 1830 3-week average records
 Getting 2-month average sales for Bishop Pauite 2 Nobi...
 Getting 3-week average sales for Pechanga | Temecula...
    ✓ 1830 3-week average records
 Gett

In [30]:
df_three_week_avg

,Date,Site,Product,Tank_1_3_Week_Avg,Tank_2_3_Week_Avg
0,2024-03-01,OLD Morongo | Cabazon,87,29734.0,NaN
1,2024-03-01,OLD Morongo | Cabazon,91,5899.0,NaN
2,2024-03-01,OLD Morongo | Cabazon,dsl,4792.0,NaN
3,2024-03-02,OLD Morongo | Cabazon,87,32464.0,NaN
4,2024-03-02,OLD Morongo | Cabazon,91,7807.0,NaN
...,...,...,...,...,...
57335,2025-10-30,Chumash,91,299.0,NaN
57336,2025-10-30,Chumash,dsl,1276.0,NaN
57337,2025-10-31,Chumash,87,4876.0,NaN
57338,2025-10-31,Chumash,91,1877.0,NaN
